In [5]:
!pip install matplotlib
!pip install seaborn
!pip install pandas
!pip install scipy

In [13]:
# Let's read the CSV file into a pandas DataFrame using the given local file path.
import pandas as pd

# Path to the CSV file
file_path = 'combined_synthetic_data_with_patient_ids.csv'

# Read the CSV file into a DataFrame
combined_df = pd.read_csv(file_path)

# Show the first few rows of the DataFrame
combined_df.head()


,Datetime,Code,Activity_Type,Heart rate___beats/minute,Calories burned_kcal,Exercise duration_s,Sleep type duration_minutes,Sleep duration_minutes,Floors climbed___floors,Patient ID
0,2022=12-08 14:17,NaN,No Physical Activity,72.0,NaN,NaN,NaN,NaN,NaN,User0
1,2022=12-08 21:05,LA11836-6,Running,156.0,20.0,40851.0,NaN,NaN,NaN,User0
2,2022=12-08 22:01,LA11836-6,Running,170.0,30.0,44878.0,NaN,NaN,NaN,User0
3,2022=12-08 22:06,LA11836-6,Running,166.0,20.0,21497.0,NaN,NaN,NaN,User0
4,2022=12-08 22:16,NaN,No Physical Activity,78.0,NaN,NaN,NaN,NaN,NaN,User0


In [23]:
!pip install ctgan
import pandas as pd
from ctgan import CTGANSynthesizer

df = combined_df

# Drop non-numeric columns or any columns that won't be synthesized
df_cleaned = df.drop(columns=['Activity_Type','Patient ID','Code', 'Datetime','Sleep type duration_minutes','Sleep duration_minutes','Floors climbed___floors'])  # Adjust according to your needs

print(df_cleaned.head())
# Fill any missing values with the column mean or other appropriate strategies
df_cleaned.fillna(df_cleaned.mean(), inplace=True)

# Instantiate the CTGAN synthesizer
synthesizer = CTGANSynthesizer()

# List of categorical columns (if any). Adjust based on your data
categorical_columns = ['Activity_Type']  # You can add more categorical columns here

# Train the CTGAN model on the data
synthesizer.fit(df_cleaned, categorical_columns=categorical_columns, epochs=300)

# Generate synthetic data (let's say you want to generate 1000 rows)
synthetic_data = synthesizer.sample(1000)

# Convert the generated data to a DataFrame
synthetic_df = pd.DataFrame(synthetic_data, columns=df_cleaned.columns)

# Save the synthetic data to a CSV file
synthetic_df.to_csv('synthetic_data_ctgan.csv', index=False)

print("Synthetic data saved to 'synthetic_data_ctgan.csv'.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 22.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 19.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 21.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 12.8 MB/s eta 0:00:00



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/fandong/opt/anaconda3/envs/BHIdatachallenge/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/fandong/opt/anaconda3/envs/BHIdatachallenge/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/fandong/opt/anaconda3/envs/BHIdatachallenge/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/fandong/opt/anaconda3/envs/BHIdatachallenge/lib/pytho

ImportError: cannot import name 'CTGANSynthesizer' from 'ctgan' (/Users/fandong/opt/anaconda3/envs/BHIdatachallenge/lib/python3.9/site-packages/ctgan/__init__.py)